In [3]:
import pandas as pd
import numpy as np

In [4]:
# Define the headers since the data does not have any
headers = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration",
           "num_doors", "body_style", "drive_wheels", "engine_location",
           "wheel_base", "length", "width", "height", "curb_weight",
           "engine_type", "num_cylinders", "engine_size", "fuel_system",
           "bore", "stroke", "compression_ratio", "horsepower", "peak_rpm",
           "city_mpg", "highway_mpg", "price"]

# Read in the CSV file and convert "?" to NaN
df = pd.read_csv("http://mlr.cs.umass.edu/ml/machine-learning-databases/autos/imports-85.data",
                  header=None, names=headers, na_values="?" )
df.head()

,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,wheel_base,...,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0
1,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0
2,1,NaN,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0
3,2,164.0,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0
4,2,164.0,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0


In [5]:
df.dtypes

symboling              int64
normalized_losses    float64
make                  object
fuel_type             object
aspiration            object
num_doors             object
body_style            object
drive_wheels          object
engine_location       object
wheel_base           float64
length               float64
width                float64
height               float64
curb_weight            int64
engine_type           object
num_cylinders         object
engine_size            int64
fuel_system           object
bore                 float64
stroke               float64
compression_ratio    float64
horsepower           float64
peak_rpm             float64
city_mpg               int64
highway_mpg            int64
price                float64
dtype: object

In [6]:
# select type 'object'
obj_col = df.select_dtypes(include=['object']).copy()
obj_col.head()

,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,engine_type,num_cylinders,fuel_system
0,alfa-romero,gas,std,two,convertible,rwd,front,dohc,four,mpfi
1,alfa-romero,gas,std,two,convertible,rwd,front,dohc,four,mpfi
2,alfa-romero,gas,std,two,hatchback,rwd,front,ohcv,six,mpfi
3,audi,gas,std,four,sedan,fwd,front,ohc,four,mpfi
4,audi,gas,std,four,sedan,4wd,front,ohc,five,mpfi


In [7]:
#Get all rows which has null value
obj_col[obj_col.isnull().any(axis=1)]

,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,engine_type,num_cylinders,fuel_system
27,dodge,gas,turbo,NaN,sedan,fwd,front,ohc,four,mpfi
63,mazda,diesel,std,NaN,sedan,fwd,front,ohc,four,idi


In [8]:
#find the value count of num_doors column
obj_col['num_doors'].value_counts()

four    114
two      89
Name: num_doors, dtype: int64

In [9]:
#replace with the most common value
obj_col = obj_col.fillna({'num_doors': 'four'})


# One Hot Encoding

In [10]:
#get dummies of "body_style", "drive_wheels" column
obj_col['body_style'].value_counts() 
pd.get_dummies(obj_col, columns=["body_style", "drive_wheels"], prefix=["body", "drive"]).head()

,make,fuel_type,aspiration,num_doors,engine_location,engine_type,num_cylinders,fuel_system,body_convertible,body_hardtop,body_hatchback,body_sedan,body_wagon,drive_4wd,drive_fwd,drive_rwd
0,alfa-romero,gas,std,two,front,dohc,four,mpfi,1,0,0,0,0,0,0,1
1,alfa-romero,gas,std,two,front,dohc,four,mpfi,1,0,0,0,0,0,0,1
2,alfa-romero,gas,std,two,front,ohcv,six,mpfi,0,0,1,0,0,0,0,1
3,audi,gas,std,four,front,ohc,four,mpfi,0,0,0,1,0,0,1,0
4,audi,gas,std,four,front,ohc,five,mpfi,0,0,0,1,0,1,0,0


In [11]:
from sklearn.preprocessing import LabelBinarizer

lb_style = LabelBinarizer()
lb_results = lb_style.fit_transform(obj_col["body_style"])
pd.DataFrame(lb_results, columns=lb_style.classes_).head()

,convertible,hardtop,hatchback,sedan,wagon
0,1,0,0,0,0
1,1,0,0,0,0
2,0,0,1,0,0
3,0,0,0,1,0
4,0,0,0,1,0


In [13]:
lb_style.classes_

array(['convertible', 'hardtop', 'hatchback', 'sedan', 'wagon'],
      dtype='<U11')

## Replace with numeric

In [10]:
#find and replace
obj_col['num_cylinders'].value_counts()

four      159
six        24
five       11
eight       5
two         4
three       1
twelve      1
Name: num_cylinders, dtype: int64

In [11]:
cleanup_nums = {"num_doors": {"four":4, "two":2},
               "num_cylinders": {"two":2 ,"three":3,"four":4 ,"six":6, "five":5, "eight": 8,"twelve":12}}
obj_col.replace(cleanup_nums, inplace=True)

In [12]:
#the replaced column's type should be changed
obj_col.dtypes

make               object
fuel_type          object
aspiration         object
num_doors           int64
body_style         object
drive_wheels       object
engine_location    object
engine_type        object
num_cylinders       int64
fuel_system        object
dtype: object

# scikit-learn
### Label Encoder

In [13]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
obj_col["make_code"] = lb_make.fit_transform(obj_col["make"])
obj_col[["make", "make_code"]].head(5)

,make,make_code
0,alfa-romero,0
1,alfa-romero,0
2,alfa-romero,0
3,audi,1
4,audi,1


## Custom binary encoding

In [14]:
#use domain knowledge to solve the problem
obj_col['engine_type'].value_counts()

ohc      148
ohcf      15
ohcv      13
dohc      12
l         12
rotor      4
dohcv      1
Name: engine_type, dtype: int64

In [15]:
obj_col["OHC_Code"] = np.where(obj_col["engine_type"].str.contains("ohc"), 1,0)

In [16]:
obj_col[['make','engine_type','OHC_Code']].head()

,make,engine_type,OHC_Code
0,alfa-romero,dohc,1
1,alfa-romero,dohc,1
2,alfa-romero,ohcv,1
3,audi,ohc,1
4,audi,ohc,1


In [17]:
obj_col['body_style'].value_counts()

sedan          96
hatchback      70
wagon          25
hardtop         8
convertible     6
Name: body_style, dtype: int64